In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip --quiet install onnx onnxruntime onnxsim
!pip install onnx-tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.

In [ ]:
!pip install flask-restful

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=b069c6d0bb1a51e398f44ba94402ce82e66525ec6287694dd7ec20dc84692d06
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [ ]:
!mkdir flask_app

In [ ]:
%cd flask_app

/content/flask_app


In [ ]:
!touch app.py

# Copy the code below into **app.py**

In [ ]:
'''from flask import Flask, request,jsonify
from flask_restful import Api, Resource
from pyngrok import ngrok
import cv2
import time
import requests
import random
import numpy as np
import onnxruntime as ort
from PIL import Image
from pathlib import Path
from collections import OrderedDict,namedtuple
import matplotlib.pyplot as plt
from numpy.lib.type_check import imag
import tensorflow as tf
cuda = False
plate_shape=(704,704)
char_shape=(288,288)
def letterbox(im, new_shape, color=(114, 114, 114), auto=True, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    # Compute padding
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding

    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, r, (dw, dh)
def init(img,classes,model,shape):
  providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if cuda else ['CPUExecutionProvider']
  session = ort.InferenceSession(model, providers=providers)
  im,r,(dw,dh)=letterbox(img,shape,auto=False)    # ex shape -> (704,704)
  #Name of the classes according to class indices.
  names = classes
  #Creating random colors for bounding box visualization.
  colors = {name:[int(random.randint(0, 255) * 0.2) for _ in range(3)] for i,name in enumerate(names)}
  return providers,session,im,r,(dw,dh),names,colors
def sortFn(li):
  return li[0]
def getCharacters(img):
  classes=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20',
         '21','22','23','24','25','26']
  model= "/content/gdrive/MyDrive/bestLetters.onnx"
  #img = cv2.imread(image_path)
  providers,session,im,r,(dw,dh),names,colors=init(img,classes,model,char_shape)
  mg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  #Preprocessing the image for prediction.
  image = img.copy()
  image, ratio, dwdh = letterbox(image,char_shape, auto=False)
  image = image.transpose((2, 0, 1))
  image = np.expand_dims(image, 0)
  image = np.ascontiguousarray(image)

  im = image.astype(np.float32)
  im /= 255
  im.shape

  #Getting onnx graph input and output names.
  outname = [i.name for i in session.get_outputs()]
  inname = [i.name for i in session.get_inputs()]
  inp = {inname[0]:im}

  # Running inference using session.
  outputs = session.run(outname, inp)[0]

  ori_images = [img.copy()]

  labels=['أ','ب','ج','د','ر','س','ص','ط','ع','ف','ق','ل','م','ن','ھ','و','ى','٠','١','٢','٣','٤', '٥', '٦', '٧', '٨','٩']
  dict_label={}
  for char in labels:
    dict_label[labels.index(char)]=char

  number=[]
  #Visualizing bounding box prediction.
  for i,(batch_id,x0,y0,x1,y1,cls_id,score) in enumerate(outputs):
      image = ori_images[int(batch_id)]
      box = np.array([x0,y0,x1,y1])
      box -= np.array(dwdh*2)
      box /= ratio
      box = box.round().astype(np.int32).tolist()
      box_coordinates=box.copy()
      cls_id = int(cls_id)
      score = round(float(score),3)
      name = names[cls_id]
      li=box.copy()
      li.append(dict_label[int(name)])
      number.append(li)
      #color = colors[name]
      #name += ' '+str(score)
      #cv2.rectangle(image,box[:2],box[2:],color,2)
      #cv2.putText(image,name,(box[0], box[1] - 2),cv2.FONT_HERSHEY_SIMPLEX,0.75,color,thickness=2)

  #plt.figure(dpi=150)
  #plt.imshow(ori_images[0])
  number.sort(key=sortFn)
  return  number

#Classification model
def predict_vehicle_class(img):
  model = tf.keras.models.load_model('/content/gdrive/MyDrive/car_classifier_VG16.h5')
  class_name={0:'bus',1:'car', 2:'large_truck', 3:'microbus', 4:'mini_bus', 5:'semi_truck', 6:'truck'}
  im=img.copy()
  im= np.array(Image.fromarray(img).resize((224, 224)))
  # expand the dimensions of the array to match the input shape of the model
  img_array = np.expand_dims(im, axis=0)
  # preprocess the image data to match the preprocessing used during training
  img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
  # use the model to make a prediction on the image
  prediction = model.predict(img_array)
  return class_name[np.argmax(prediction)]

def model(img):
  classes=['License Plate']
  model= "/content/gdrive/MyDrive/bestPlate.onnx"
  #img = cv2.imread(image_path)
  providers,session,im,r,(dw,dh),names,colors=init(img,classes,model,plate_shape)
  for_croped = img.copy()
  mg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  #Preprocessing the image for prediction.
  image = img.copy()
  image, ratio, dwdh = letterbox(image,plate_shape, auto=False)
  image = image.transpose((2, 0, 1))
  image = np.expand_dims(image, 0)
  image = np.ascontiguousarray(image)

  im = image.astype(np.float32)
  im /= 255
  im.shape

  #Getting onnx graph input and output names.
  outname = [i.name for i in session.get_outputs()]
  inname = [i.name for i in session.get_inputs()]
  inp = {inname[0]:im}

  # Running inference using session.
  outputs = session.run(outname, inp)[0]

  ori_images = [img.copy()]
  box_coordinates=[]
  #Visualizing bounding box prediction.
  for i,(batch_id,x0,y0,x1,y1,cls_id,score) in enumerate(outputs):
      image = ori_images[int(batch_id)]
      box = np.array([x0,y0,x1,y1])
      box -= np.array(dwdh*2)
      box /= ratio
      box = box.round().astype(np.int32).tolist()
      box_coordinates=box.copy()
      cls_id = int(cls_id)
      score = round(float(score),3)
      name = names[cls_id]
      #color = colors[name]
      name += ' '+str(score)
      #cv2.rectangle(image,box[:2],box[2:],color,2)
      x, y, w, h = box[0], box[1], box[2]-box[0], box[3]-box[1]
      for_croped=for_croped[y:y+h, x:x+w]
      #cv2.putText(image,name,(box[0], box[1] - 2),cv2.FONT_HERSHEY_SIMPLEX,0.75,color,thickness=2)
  plt.figure(dpi=150)
  plt.imshow(ori_images[0])
  number=getCharacters(for_croped)
  _type=predict_vehicle_class(img)
  return number,_type

app = Flask(__name__)
api = Api(app)
class YOLOv7API(Resource):
    def post(self):
      # Get the image from the request
      #image = request.files['image'].read()
      #image = np.frombuffer(image, dtype='uint8')
      img = Image.open(request.files['image'])
      image= np.array(img)
      boxes,_type = model(image)
      results = []
      for i in range(len(boxes)):
        results.append({
             'box': boxes[i][:4],
             'class': boxes[i][4]
           })
      return dict({'License Number': results,'Vehicle Class':_type})
api.add_resource(YOLOv7API, '/predict',methods=['GET', 'POST'])
app.config['PROPAGATE_EXCEPTIONS'] = True
if __name__ == '__main__':
    #url = ngrok.connect(5000).public_url
    url=ngrok.connect(5000, 'http', ngrok_address='http://56fc-34-86-75-112.ngrok.io')
    print(' * Tunnel URL:', url)
    # Start the Flask app and listen for requests
    app.run(debug=False,host='0.0.0.0', port=5000)
'''

'from flask import Flask, request,jsonify\nfrom flask_restful import Api, Resource\nfrom pyngrok import ngrok\nimport cv2\nimport time\nimport requests\nimport random\nimport numpy as np\nimport onnxruntime as ort\nfrom PIL import Image\nfrom pathlib import Path\nfrom collections import OrderedDict,namedtuple\nimport matplotlib.pyplot as plt\nfrom numpy.lib.type_check import imag\nimport tensorflow as tf\ncuda = False\nplate_shape=(704,704)\nchar_shape=(288,288)\ndef letterbox(im, new_shape, color=(114, 114, 114), auto=True, scaleup=True, stride=32):\n    # Resize and pad image while meeting stride-multiple constraints\n    shape = im.shape[:2]  # current shape [height, width]\n    if isinstance(new_shape, int):\n        new_shape = (new_shape, new_shape)\n\n    # Scale ratio (new / old)\n    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])\n    if not scaleup:  # only scale down, do not scale up (for better val mAP)\n        r = min(r, 1.0)\n\n    # Compute padding\n    new_u

In [ ]:
!python3 app.py

2023-02-20 20:18:15.673700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 20:18:16.960047: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-20 20:18:16.960185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist